In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np

import torchvision.transforms as transforms 

ModuleNotFoundError: No module named 'robustnessgym.core.cells'

In [1]:
from terra import Task

In [4]:
from spr.data.celeb import build_celeb_df
df = build_celeb_df(dataset_dir="/Users/sabrieyuboglu/data/datasets/celeba")

task: build_celeb_df, run_id=5


In [6]:
df.__dict__

{'run_dir': '/Users/sabrieyuboglu/code/terra/test_storage_dir/tasks/data/celeb/build_celeb_df/_runs/5',
 'key': 'c1a2dccd37dc409b9947d9fe3e3b9578',
 'id': 4,
 'type': pandas.core.frame.DataFrame}

In [9]:
csv_path = "/Users/sabrieyuboglu/code/terra/test_storage_dir/tasks/data/celeb/build_celeb_df/_runs/5/artifacts/c1a2dccd37dc409b9947d9fe3e3b9578"
df = pd.read_csv(csv_path)
dp = DataPane.from_panas(df)

NameError: name 'DataPane' is not defined

In [7]:
# create cell column
from torch.utils.data._utils.collate import default_collate 
csv_path = "/Users/sabrieyuboglu/code/terra/test_storage_dir/tasks/data/celeb/build_celeb_df/_runs/5/artifacts/c1a2dccd37dc409b9947d9fe3e3b9578"

cols = {
    "name": list(np.arange(20)),
    "id": list(np.arange(20) * 10),
}

data_dir = "/Users/sabrieyuboglu/data/datasets/celeba/img_align_celeba"
cells = [
    ImagePath(
        filepath=os.path.join(data_dir, filename),
        transform=transforms.PILToTensor()
    )
    for filename in 
    list(os.listdir(data_dir))[:20]
]
cols["img"] = CellColumn.from_cells(
    cells, 
    collate_fn=default_collate,
    materialize=True
)

# spacy column 
from robustnessgym.core.cells.spacy import SpacyCell
import spacy
nlp = spacy.load("en_core_web_sm")
cells = [
    SpacyCell(
        doc=nlp(os.path.join(data_dir, text)),
    )
    for text in list(os.listdir(data_dir))[:20]
]
cols["doc"] = CellColumn.from_cells(cells, materialize=False)

# create the dataset
from robustnessgym.core.new_dataset import Dataset
dataset = Dataset.from_batch(cols)

In [12]:
state = cols["img"].get_state()
out = CellColumn.from_state(state)
out[0]

tensor([[[ 77,  59,  47,  ...,  45,  43,  40],
         [ 87,  69,  56,  ...,  47,  43,  40],
         [102,  90,  76,  ...,  45,  42,  39],
         ...,
         [ 28,  23,  30,  ...,  15,  14,  14],
         [ 26,  23,  36,  ...,  13,  17,  17],
         [ 41,  29,  25,  ...,  13,  17,  17]],

        [[ 89,  71,  59,  ...,  62,  43,  40],
         [ 96,  81,  68,  ...,  62,  43,  40],
         [111,  99,  85,  ...,  59,  43,  40],
         ...,
         [ 29,  27,  36,  ...,  16,  14,  14],
         [ 27,  27,  42,  ...,  17,  17,  17],
         [ 42,  33,  31,  ...,  17,  17,  17]],

        [[ 43,  25,  13,  ...,  69,  51,  48],
         [ 51,  35,  22,  ...,  69,  51,  48],
         [ 64,  52,  40,  ...,  68,  48,  45],
         ...,
         [ 34,  30,  36,  ...,  18,  14,  14],
         [ 32,  30,  42,  ...,  18,  17,  17],
         [ 47,  36,  31,  ...,  18,  17,  17]]], dtype=torch.uint8)

In [22]:
state = cols["doc"].get_state()
out = CellColumn.from_state(state, nlp=nlp)
out._cells[0]

/Users/sabrieyuboglu/data/datasets/celeba/img_align_celeba/052628.jpg

In [30]:
out = dataset.filter(
    batch_size=2, function=lambda x: (x["name"] % 2) == 0, batched=False
)


 91%|█████████ | 10/11 [00:00<00:00, 187.12it/s]


In [21]:
out = dataset.update(
    batch_size=2, 
    function=lambda x: {
        "is_even": (x["name"] % 2) == 0,
        "is_odd": (x["name"] % 2) == 1,
        "image_path": ImagePath(str(x["doc"]))
    }, 
    batched=False
)


 91%|█████████ | 10/11 [00:00<00:00, 183.46it/s]


In [24]:
dataset["doc"]

In [126]:
for x in dl:
    pass

In [74]:
def nested_map(f, *args):
    """ Recursively transpose a nested structure of tuples, lists, and dicts """
    assert len(args) > 0, 'Must have at least one argument.'
    arg = args[0]
    if isinstance(arg, Sequence):
        return [nested_map(f, *a) for a in zip(*args)]
    elif isinstance(arg, Mapping):
        return {
            k: nested_map(f, *[a[k] for a in args])
            for k in arg
        }
    else:
        return f(*args)

In [75]:
def f(obj):
    if hasattr(obj, "get_state"):
        return State({
            "class": type(obj),
            "state": obj.get_state()
        })
    else:
        return obj 

def f_prime(obj):
    if isinstance(obj, State):
        return obj.class.from_state(obj.state)
    else:
        return obj
nested_map(lambda x: x + 1, {"a": [1,2,3], "b": 6, "c": [{"a": 1, "b": 2}, {"a": 2, "b": 3}]})

{'a': [2, 3, 4], 'b': 7, 'c': [{'a': 2, 'b': 3}, {'a': 3, 'b': 4}]}

In [57]:
col._materialize = True
def fn(batch):
    batch = batch.to(float)
    return list(map(float, batch.mean(dim=(3, 2, 1))))
col.map(fn, batched=True, batch_size=4)

100%|██████████| 5/5 [00:00<00:00, 54.01it/s][117.96048517334982, 133.02053052949867] 117.96048517334982
[]
[117.96048517334982, 133.02053052949867, 148.65947668625228, 123.81946878328695]
[64.49006116207951, 79.93583135759201, 150.21430780331926, 170.99054221214308]
[87.44167267979246, 71.77779782153043, 136.6673710614026, 92.9854568257568]
[134.84880424698486, 106.93297941792942, 131.30221111225646, 101.4897948665086]
[118.16871971961653, 168.44098546541593, 127.06906504484074, 113.39196990001031]



[117.96048517334982,
 133.02053052949867,
 148.65947668625228,
 123.81946878328695,
 64.49006116207951,
 79.93583135759201,
 150.21430780331926,
 170.99054221214308,
 87.44167267979246,
 71.77779782153043,
 136.6673710614026,
 92.9854568257568,
 134.84880424698486,
 106.93297941792942,
 131.30221111225646,
 101.4897948665086,
 118.16871971961653,
 168.44098546541593,
 127.06906504484074,
 113.39196990001031]

In [26]:
col._materialize = True
def filter_fn(batch):
    batch = batch.to(float)
    return batch.mean(dim=(1,2,3)) > 100

out = col.filter(filter_fn, batched=True, batch_size=4)

100%|██████████| 5/5 [00:00<00:00, 61.34it/s]


In [27]:
len(out)

15

In [19]:
out = col.map(filter_fn, batched=True, batch_size=4)
out

100%|██████████| 5/5 [00:00<00:00, 61.16it/s]


[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [31]:
col.write("test_col", write_together=True)

In [32]:
out = CellColumn.read("test_col")

In [25]:
out._materialize=False
out[0].transform

<function torchvision.transforms.functional.pil_to_tensor(pic)>

In [ ]:
def __getitem__() -> VisionColumn:

In [10]:
from torch import Tensor

In [18]:
from yaml.representer import Representer
from abc import ABCMeta
Representer.add_representer(ABCMeta, Representer.represent_name)

In [59]:
bool(15)

True

In [61]:
class TestState:
    def __init__(self):
        self.x = np.ones(10) * 5
        self.y = np.ones(10) * 10

    
    def __getstate__(self):
        return self
        
    def __setstate__(self, state):
        self.__dict__ = state.__dict__
        

In [62]:
import pickle 
import dill
obj = TestState()
obj.x = np.ones(10) * 50
obj.y = np.ones(10) * 100

dill.dump(obj, open("test.pkl", "wb"))

In [63]:
new_obj = dill.load(open("test.pkl", "rb"))

In [64]:
new_obj.__dict__

{}